In [4]:
# 格式： 
# SMILES,pKa_num, pKa_value
# NCC(=O)O, 2, "3,9"
import pandas as pd
import numpy as np
import os

# --- Configuration ---
INPUT_CSV_PATH = '../data/NIST_database_onlyH_6TypeEq_pos_match_max_fg_other.csv'
OUTPUT_CSV_PATH = '../data/processed_pka_data.csv'
OUTPUT_COLUMNS = ['SMILES', 'pKa_num', 'pKa_value']

# --- Main Processing Logic ---
def process_pka_data(input_path: str, output_path: str):
    """
    Reads the NIST pKa data, averages values for each equilibrium per SMILES,
    and creates a new CSV with max_eq_num and sorted, comma-separated pKa values.
    """
    print(f"Reading input CSV: {input_path}")
    try:
        df = pd.read_csv(input_path)
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_path}")
        return
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return

    # --- Validate required columns ---
    required_cols = ['SMILES', 'Equilibrium', 'Value', 'max_eq_num']
    if not all(col in df.columns for col in required_cols):
        print(f"Error: Input CSV missing one or more required columns: {required_cols}")
        return

    # --- Data Cleaning (Optional but recommended) ---
    # Convert 'Value' to numeric, coercing errors to NaN
    df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
    # Drop rows where averaging is impossible (missing SMILES or Value)
    original_rows = len(df)
    df.dropna(subset=['SMILES', 'Value'], inplace=True)
    if len(df) < original_rows:
        print(f"Dropped {original_rows - len(df)} rows with missing SMILES or non-numeric Value.")

    print("Step 1: Averaging pKa values for each SMILES/Equilibrium pair...")
    # Group by SMILES and the specific equilibrium, then average the 'Value'
    df_averaged = df.groupby(['SMILES', 'Equilibrium'], as_index=False)['Value'].mean()
    print(f"Averaged data shape: {df_averaged.shape}")

    print("Step 2: Aggregating averaged pKa values per SMILES...")
    # Group again just by SMILES to collect all averaged pKa values for that molecule
    # Apply list to get all values, then apply sorting and formatting
    def aggregate_pka(series):
        # Sort numerically
        sorted_pka = sorted(series.tolist())
        # Format to string with 2 decimal places, separated by comma
        return ",".join([f"{pka:.2f}" for pka in sorted_pka])

    df_aggregated_pka = df_averaged.groupby('SMILES')['Value'].apply(aggregate_pka).reset_index()
    df_aggregated_pka.rename(columns={'Value': 'pKa_value'}, inplace=True)
    print(f"Aggregated pKa data shape: {df_aggregated_pka.shape}")

    print("Step 3: Getting max_eq_num for each SMILES...")
    # Get the max_eq_num (should be consistent per SMILES, take the first)
    # Ensure max_eq_num is integer where possible
    df_max_eq = df.groupby('SMILES', as_index=False)['max_eq_num'].first()
    # Attempt conversion to integer, keeping floats if necessary (e.g., if NaNs existed)
    try:
        df_max_eq['max_eq_num'] = df_max_eq['max_eq_num'].astype(int)
    except (ValueError, TypeError):
        print("Warning: Could not convert all 'max_eq_num' values to integers.")
    df_max_eq.rename(columns={'max_eq_num': 'pKa_num'}, inplace=True)


    print("Step 4: Merging results...")
    # Merge the aggregated pKa strings with the max_eq_num
    df_final = pd.merge(df_max_eq, df_aggregated_pka, on='SMILES', how='inner')

    # Ensure correct column order
    df_final = df_final[OUTPUT_COLUMNS]
    df_final = df_final.sort_values(by='pKa_num')
    print(f"Final processed data shape: {df_final.shape}")

    # --- Write Output ---
    try:
        # Ensure output directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        print(f"Writing output CSV: {output_path}")
        df_final.to_csv(output_path, index=False) # quoting=3 for csv.QUOTE_NONE
        print("Processing complete.")
    except Exception as e:
        print(f"Error writing output CSV: {e}")

# --- Run the processing ---
if __name__ == "__main__":
    process_pka_data(INPUT_CSV_PATH, OUTPUT_CSV_PATH)


Reading input CSV: ../data/NIST_database_onlyH_6TypeEq_pos_match_max_fg_other.csv
Step 1: Averaging pKa values for each SMILES/Equilibrium pair...
Averaged data shape: (3562, 3)
Step 2: Aggregating averaged pKa values per SMILES...
Aggregated pKa data shape: (1958, 2)
Step 3: Getting max_eq_num for each SMILES...
Step 4: Merging results...
Final processed data shape: (1958, 3)
Writing output CSV: ../data/processed_pka_data.csv
Processing complete.
